In [1]:
import tensorflow as tf

/home/jugs/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jugs/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jugs/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jugs/anaconda3/envs/myenv/lib/python3.6/site-pack

In [2]:
export_dir = "/tmp/tfmodeldir/1"

In [3]:
input_placeholder = tf.placeholder(tf.int32, shape=[None, 1], name='input_placeholder')
output_placeholder = tf.identity(tf.add(input_placeholder, 2), name='output_placeholder')

In [4]:
sess = tf.Session()
tf.saved_model.simple_save(sess, export_dir, inputs={"key":input_placeholder}, outputs={"keys":output_placeholder})

W1023 15:56:41.153825 139813060958016 deprecation.py:323] From <ipython-input-4-6ed9f0a7c3a6>:2: simple_save (from tensorflow.python.saved_model.simple_save) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
W1023 15:56:41.154382 139813060958016 deprecation.py:323] From /home/jugs/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/saved_model/signature_def_utils_impl.py:201: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


AssertionError: Export directory already exists, and isn't empty. Please choose a different export directory, or delete all the contents of the specified directory: /tmp/tfmodeldir/1

In [5]:
!tree "/tmp/tfmodeldir/1"

/tmp/tfmodeldir/1
├── saved_model.pb
└── variables

1 directory, 1 file


# Checking the saved model signature

In [6]:
!saved_model_cli show --dir /tmp/tfmodeldir/1 --all

/home/jugs/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jugs/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jugs/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jugs/anaconda3/envs/myenv/lib/python3.6/site-pack

# Tensorflow Serving

In [ ]:
!tensorflow_model_server --port=8500 --rest_api_port=8501 --model_name=simple_add --model_base_path=/tmp/tfmodeldir/

2019-10-23 01:32:25.761732: I tensorflow_serving/model_servers/server.cc:82] Building single TensorFlow model file config:  model_name: simple_add model_base_path: /tmp/tfmodeldir/
2019-10-23 01:32:25.761842: I tensorflow_serving/model_servers/server_core.cc:462] Adding/updating models.
2019-10-23 01:32:25.761853: I tensorflow_serving/model_servers/server_core.cc:561]  (Re-)adding model: simple_add
2019-10-23 01:32:25.862420: I tensorflow_serving/core/basic_manager.cc:739] Successfully reserved resources to load servable {name: simple_add version: 1}
2019-10-23 01:32:25.862486: I tensorflow_serving/core/loader_harness.cc:66] Approving load for servable version {name: simple_add version: 1}
2019-10-23 01:32:25.862515: I tensorflow_serving/core/loader_harness.cc:74] Loading servable version {name: simple_add version: 1}
2019-10-23 01:32:25.862566: I external/org_tensorflow/tensorflow/contrib/session_bundle/bundle_shim.cc:363] Attempting to load native SavedModelBundle in bundle-shim from

# Client for inference

In [ ]:
import requests
import json

endpoint = "http://localhost:8501/v1/models/simple_add:predict"

instance = [[11,1],[22,1],[44,1],[55,1]]


headers = {"content-type":"application-json"}
data = json.dumps({"signature-name":"serving_default", "instances": instance})
response = requests.post(endpoint, data=data, headers=headers)
prediction = json.loads(response.text)['predictions']
prediction

In [13]:
# with tf.gfile.GFile("/tmp/tfmodeldir/1/saved_model.pb", "rb") as f:
#     graph_def = tf.GraphDef()
#     graph_def.ParseFromString(f.read())

     
# with tf.Graph().as_default() as graph:
#       tf.import_graph_def(graph_def,
#                           input_map=None,
#                           return_elements=None,
#                           name=""
#       )
        
        
frozen_graph="/tmp/tfmodeldir/1/saved_model.pb"

with tf.gfile.GFile(frozen_graph, "rb") as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

IndentationError: unexpected indent (<ipython-input-13-0257f920ab98>, line 5)

In [ ]:
with tf.Graph().as_default() as graph:
      tf.import_graph_def(graph_def,
                          input_map=None,
                          return_elements=None,
                          name=""
      )

# Freezing model from the Keras 

In [24]:
import keras
import json
import os
import sys
import tensorflow as tf
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input

In [30]:
base_path = "/tmp/tfmodeldir/kerasmodel"

In [31]:
inception_model = InceptionV3(weights='imagenet', input_tensor=Input(shape=(299, 299, 3)))
inception_model.save(os.path.join(base_path,'inception.h5'))

In [36]:
#tf.keras.backend.set_learning_phase(0)

model = tf.keras.models.load_model(os.path.join(base_path, 'inception.h5'))
model.summary()

RuntimeError: Attempted to use a closed Session.

E1023 13:36:36.233739 140002600597312 tf_should_use.py:71] ==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'VarIsInitializedOp_755:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/jugs/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3346, in run_code
    return outflag  File "<ipython-input-21-0c32bb0a1afa>", line 7, in <module>
    model = tf.keras.models.load_model(os.path.join(keras_model, "inception.h5"))  File "/home/jugs/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/keras/saving/save.py", line 146, in load_model
    return hdf5_format.load_model_from_hdf5(filepath, custom_objects, compile)  File "/home/jugs/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/keras/saving/hdf5_format.py", line 270, in load_model_from_hdf5
    f.close()  File "/home/jugs/anaconda

E1023 13:36:36.236920 140002600597312 tf_should_use.py:71] ==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'VarIsInitializedOp_751:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/jugs/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3346, in run_code
    return outflag  File "<ipython-input-21-0c32bb0a1afa>", line 7, in <module>
    model = tf.keras.models.load_model(os.path.join(keras_model, "inception.h5"))  File "/home/jugs/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/keras/saving/save.py", line 146, in load_model
    return hdf5_format.load_model_from_hdf5(filepath, custom_objects, compile)  File "/home/jugs/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/keras/saving/hdf5_format.py", line 270, in load_model_from_hdf5
    f.close()  File "/home/jugs/anaconda

E1023 13:36:36.240278 140002600597312 tf_should_use.py:71] ==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'VarIsInitializedOp_747:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/jugs/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3346, in run_code
    return outflag  File "<ipython-input-21-0c32bb0a1afa>", line 7, in <module>
    model = tf.keras.models.load_model(os.path.join(keras_model, "inception.h5"))  File "/home/jugs/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/keras/saving/save.py", line 146, in load_model
    return hdf5_format.load_model_from_hdf5(filepath, custom_objects, compile)  File "/home/jugs/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/keras/saving/hdf5_format.py", line 270, in load_model_from_hdf5
    f.close()  File "/home/jugs/anaconda

E1023 13:36:36.243652 140002600597312 tf_should_use.py:71] ==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'VarIsInitializedOp_743:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/jugs/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3346, in run_code
    return outflag  File "<ipython-input-21-0c32bb0a1afa>", line 7, in <module>
    model = tf.keras.models.load_model(os.path.join(keras_model, "inception.h5"))  File "/home/jugs/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/keras/saving/save.py", line 146, in load_model
    return hdf5_format.load_model_from_hdf5(filepath, custom_objects, compile)  File "/home/jugs/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/keras/saving/hdf5_format.py", line 270, in load_model_from_hdf5
    f.close()  File "/home/jugs/anaconda

E1023 13:36:36.247118 140002600597312 tf_should_use.py:71] ==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'VarIsInitializedOp_739:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/jugs/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3346, in run_code
    return outflag  File "<ipython-input-21-0c32bb0a1afa>", line 7, in <module>
    model = tf.keras.models.load_model(os.path.join(keras_model, "inception.h5"))  File "/home/jugs/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/keras/saving/save.py", line 146, in load_model
    return hdf5_format.load_model_from_hdf5(filepath, custom_objects, compile)  File "/home/jugs/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/keras/saving/hdf5_format.py", line 270, in load_model_from_hdf5
    f.close()  File "/home/jugs/anaconda

...

E1023 13:36:36.530225 140002600597312 tf_should_use.py:71] ==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'VarIsInitializedOp_391:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/jugs/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3346, in run_code
    return outflag  File "<ipython-input-21-0c32bb0a1afa>", line 7, in <module>
    model = tf.keras.models.load_model(os.path.join(keras_model, "inception.h5"))  File "/home/jugs/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/keras/saving/save.py", line 146, in load_model
    return hdf5_format.load_model_from_hdf5(filepath, custom_objects, compile)  File "/home/jugs/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/keras/saving/hdf5_format.py", line 270, in load_model_from_hdf5
    f.close()  File "/home/jugs/anaconda

E1023 13:36:36.532641 140002600597312 tf_should_use.py:71] ==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'VarIsInitializedOp_387:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/jugs/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3346, in run_code
    return outflag  File "<ipython-input-21-0c32bb0a1afa>", line 7, in <module>
    model = tf.keras.models.load_model(os.path.join(keras_model, "inception.h5"))  File "/home/jugs/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/keras/saving/save.py", line 146, in load_model
    return hdf5_format.load_model_from_hdf5(filepath, custom_objects, compile)  File "/home/jugs/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/keras/saving/hdf5_format.py", line 270, in load_model_from_hdf5
    f.close()  File "/home/jugs/anaconda

E1023 13:36:36.535320 140002600597312 tf_should_use.py:71] ==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'VarIsInitializedOp_383:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/jugs/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3346, in run_code
    return outflag  File "<ipython-input-21-0c32bb0a1afa>", line 7, in <module>
    model = tf.keras.models.load_model(os.path.join(keras_model, "inception.h5"))  File "/home/jugs/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/keras/saving/save.py", line 146, in load_model
    return hdf5_format.load_model_from_hdf5(filepath, custom_objects, compile)  File "/home/jugs/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/keras/saving/hdf5_format.py", line 270, in load_model_from_hdf5
    f.close()  File "/home/jugs/anaconda

E1023 13:36:36.538147 140002600597312 tf_should_use.py:71] ==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'VarIsInitializedOp_379:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/jugs/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3346, in run_code
    return outflag  File "<ipython-input-21-0c32bb0a1afa>", line 7, in <module>
    model = tf.keras.models.load_model(os.path.join(keras_model, "inception.h5"))  File "/home/jugs/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/keras/saving/save.py", line 146, in load_model
    return hdf5_format.load_model_from_hdf5(filepath, custom_objects, compile)  File "/home/jugs/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/keras/saving/hdf5_format.py", line 270, in load_model_from_hdf5
    f.close()  File "/home/jugs/anaconda

KeyboardInterrupt: 

In [27]:
tf.keras.backend.set_learning_phase(0)
model = tf.keras.models.load_model(os.path.join(base_path, "inception.h5"))

export_dir = os.path.join(keras_model, '1')


# with tf.keras.backend.get_session() as sess:
#     tf.saved_model.simple_save(sess, 
#                               export_dir, 
#                               inputs= {"keys":model.input},
#                               outputs= {t.name: t for t in model.outputs})


model.summary()

# with tf.keras.backend.get_session() as sess:
#     tf.saved_model.simple_save(
#         sess,
#         export_dir,
#         inputs={'input_image': model.input},
#         outputs={t.name: t for t in model.outputs})

KeyError: "Unable to open object (object 'activation_392' doesn't exist)"